# Проверка гипотезы по автоматизации проверки данных DataSet на предмет "подозрительных" выбросов занчений данных

 Мы провели графический анализ данных (см. Diploma_1). Исследование  данных в столбцах по типу box-and-wisker plot (ящик с усами) наглядно поазывают ,что в ряде столбцов ,таких как:
Шаг нашивки                       
Плотность нашивки                 
модуль упругости, ГПа            
Количество отвердителя, м.%     
Поверхностная плотность, г/м2   
Потребление смолы, г/м2
имеются выбросы как в минимальной области (даже включя ноль, либо значения близкие к нулю - что противоречит физическим принципам)
Либо значения на порядок, а то и на 3 поядка меньше медианы данных конкретного столбца так  и в максимальной области.

Для принятия решения по изменению конкретных данных (замена на медиану или на умножение на конкретный коэффициент) или удалению конкретной строки данных в случае сильной флуктуации данных в строке принимаем гипотезу по созданию матрицы "подозрительных данных" для дальнейшей аналиа.

In [ ]:
# Импортируем в проект требуемые библиотеки

import seaborn as sns # библиотека для создания статистических графиков
import random #  генераторатор случайных чисел и данных
import pandas as pd # библиотека для обработки и анализа данных
import os # библиотека функций для работы с операционной системой.
import numpy as np
import tensorflow as tf # фреймворк для глубокого машинного обучения
import csv

#!pip install googletrans==3.1.0a0
#  взято из https://www.techgeekbuzz.com/blog/how-to-translate-languages-in-python/
#!pip install googletrans==4.0.0rc1 


import googletrans
from googletrans import Translator

#!pip google_trans_new
#from google_trans_new import google_translator  
  



#import matplotlib.pyplot as plt
#from matplotlib import cm
#from mpl_toolkits.mplot3d import Axes3D
#import matplotlib.pyplot as plt

#import plotly
#import plotly.graph_objs as go
#import plotly.express as px
#from plotly.subplots import make_subplots

Collecting googletrans==4.0.0rc1
  Downloading googletrans-4.0.0rc1.tar.gz (20 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Requirement already satisfied: httpx==0.13.3 in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from googletrans==4.0.0rc1) (0.13.3)
Requirement already satisfied: chardet==3.* in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (3.0.4)
Requirement already satisfied: rfc3986<2,>=1.3 in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (1.5.0)
Requirement already satisfied: idna==2.* in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (2.10)
Requirement already satisfied: hstspreload in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (2023.1.1)
Requirement already satisfied: certifi in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (2022.12.7)
Requirement already satisfied: sniffio in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (1.2.0)
Requirement already satisfied: httpcore==0.9.* in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpx==0.13.3->googletrans==4.0.0rc1) (0.9.1)
Requirement already satisfied: h2==3.* in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpcore==0.9.*->httpx==0.13.3->googletrans==4.0.0rc1) (3.2.0)
Requirement already satisfied: h11<0.10,>=0.8 in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from httpcore==0.9.*->httpx==0.13.3->googletrans==4.0.0rc1) (0.9.0)
Requirement already satisfied: hyperframe<6,>=5.2.0 in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from h2==3.*->httpcore==0.9.*->httpx==0.13.3->googletrans==4.0.0rc1) (5.2.0)
Requirement already satisfied: hpack<4,>=3.0 in c:\users\grain\anaconda3\envs\myenv_jupiterlab\lib\site-packages (from h2==3.*->httpcore==0.9.*->httpx==0.13.3->googletrans==4.0.0rc1) (3.0.0)
Building wheels for collected packages: googletrans
  Building wheel for googletrans (setup.py): started
  Building wheel for googletrans (setup.py): finished with status 'done'
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17411 sha256=bc4c2f0f85c65f2d2a2e6b23c6c63a34ef82fb5b700350b263be158517ef0a76
  Stored in directory: c:\users\grain\appdata\local\pip\cache\wheels\54\ca\27\562b6eac3a495887e4b44bac3a1efe925fa603d085ba89a21d
Successfully built googletrans
Installing collected packages: googletrans
  Attempting uninstall: googletrans
    Found existing installation: googletrans 3.1.0a0
    Uninstalling googletrans-3.1.0a0:
      Successfully uninstalled googletrans-3.1.0a0
Successfully installed googletrans-4.0.0rc1

# Создадим матрицу(массив) "подозрительных данных"

In [ ]:
# теперь прочитаем наш файл формата hw_data_composite_new.CSV в  котором будем изменять часть данных в определенных строках

hw_data_composite_analyzis = pd.read_csv(r'C:\Users\grain\Work_folder\Diplom_MGTU\hw_data_composite_new.csv')
hw_data_composite_analyzis.info()

In [ ]:
# применим метод .describe() к количественным признакам
hw_data_composite_analyzis.describe(percentiles = [0.25, 0.5, 0.75, 0.99], include='all').round(3) # дополнительно выводим еще одну квартилю 99%

# Проверим наш DataFrame на наличие дубликатов  строк 

In [ ]:
hw_data_composite_analyzis_dupl = hw_data_composite_analyzis.loc[:,~hw_data_composite_analyzis.apply(lambda x: x.duplicated(),axis=1).all()].copy()

In [ ]:
hw_data_composite_analyzis_dupl.info()

In [ ]:
hw_data_composite_analyzis_dupl

# В нашем dataFrame НЕТ дубликатов

# Перевод  значений str из списка на другой язык (с любого )

# Заменим  названия столбцов с русского языка на английский язык

In [ ]:
# Функция перевода стhоковых данных ('слово или предложение') или Списка (list) на другой язык
# передаем параметры:
# list_data - строку в формате 'str' или в виде списка list
# lang_old - необязательный параметр - src - код  исходного языка - по таблие сокращений языков (в нашем случае - 'ru' - русский) 
# lang_new - параметр -dest -код   целевого языка - по таблице сокращений языков (по умолчанию - 'en' - английский)
# Если не указать lang_new - параметр -'dest' - прорамма по умолчанию применит английский язык.
# prt - параметр для печати результатов. Если = True - печать, если Flse - Нет печати

def translate_list_data (data, src, dest, prt = True):
    
    list_old_name = data
    # Создаем пустой список для вывод переведенного списка
    list_new_name = []

    translator = Translator() # вызываем метод Translator()

    for i in list_old_name: # проходим цикл по списку
   
        translated = translator.translate(i, src = 'ru', dest= 'en').text # осуществляем перевод по элемнтам списка
    
        list_new_name.append(translated) # добавляем переведенные элементы  в списое первода

    if prt==True:
        print(f'{list_old_name} -> {list_new_name}')
        
    return list_new_name

In [ ]:
list_column =hw_data_composite_analyzis.columns.tolist() # Cписок наименований столбцов для передачи в функцию перевода

In [ ]:
list_new_name = translate_list_data(list_column,'ru', 'en', prt = True) # Функция перевода названий столбцов ( в данном случае) 

In [ ]:
print(list_new_name)

In [ ]:
# Переименуем наименования столбцов в нашем df
# часть взята из https://dev-gang.ru/article/perevod-teksta-s-pomosczu-google-translate-api-v-python-ahgm88wx1k/

def сhangelang_column_name(df, list_new_name, prt = True):
    # функция полчает df  и  заменяет названия столбцов с русского языка на ангийский язык 
    # list_new_name - список новых наименований столбцов
    # list_old_name -  Список старых названий столбцов
  
    list_old_name = list(df.columns)
            
    # Создадим словарь для его подcтановки в метод .rename для замены имен столбцов
    dict1 = {} # создаем пустой словарь

    for i in range(len(list_old_name)):
        dict1[list_old_name[i]] = list_new_name[i]
        df.rename(columns=dict1, inplace=True)

    if prt==True:
        print(dict1)
    return df

In [ ]:
сhangelang_column_name(hw_data_composite_analyzis, list_new_name)

In [ ]:
# теперь запишем наш файл с исходными данными с измененным названием столбцов на английский языке в формате .CSV 
# в нашу папку,где находится наш проектный Jupiter_notebook-Diplom_MGTU
# и также приводим количество знаков у переменных типа float к 3 знакам после запятой float_format="%.3f"

hw_data_composite_analyzis.to_csv('composite_analyzis_eng.csv', index=False, float_format="%.3f")


# Мы перевели названия столбцов а английский язык и заменили названия с русского на английский

In [ ]:
hw_data_composite_analyzis.head(21)

# замечание - в  строке с индексом 19 в столбце 'Шаг нашивки' и 'Плотность нашивки' есть значения '0.0'.
нам необходимо провести анализ и найти 'подозрительного значения'

# Заметим, что '50% процентиль' тоже, что и медиана!!!
Источник - https://pandas.pydata.org/docs/dev/reference/api/pandas.DataFrame.describe.html

In [ ]:
# Вычислим список медиан по всем столбцам - это процентиль 50%

median_composit = hw_data_composite_analyzis.describe(include='all').loc['50%']
median_composit

# Создадим DataFrame 'median_set' из названий столбцов и значений медиан столбцов
# Источник - https://stackoverflow.com/questions/26097916/convert-pandas-series-to-dataframe

#median_set = pd.DataFrame({'Сolumns':median_composit.index, '50%-median ':median_composit.values})
#median_set # выводим на печать

 new_row_median = hw_data_composite_analyzis.median(axis=0) # параметры новой строки в конце массива - это медианы столбцов!!

 df_zeros.append(new_row_median, ignore_index = True) # создаем новую строку медиан в конце массива # работает , строка добавляется



 значение критериев 'min_data'  - все значения меньше 10% от медианы- 'подозрительного значения'
 значение критериев 'max_data'  - все значения больше медианы в 10 раз  - 'подозрительного значения'
min_data = 0.1
max_data = 10.0
 
 переводим список медиан в DataFrame
 df_median = pd.DataFrame({'Сolumns':median_composit.index, '50%-median ':median_composit.values}) 

for column in hw_data_composite_analyzis.columns: #
    for i in range(len(hw_data_composite_analyzis)):
        
        value = hw_data_composite_analyzis.at[i,column] # значение из df -  hw_data_composite_analyzis ВАЖНО присвоение .at[i,column]
        min_median = min_data * median_composit[column] # значение критерия 'min_median' = критерий min * медиану столбца
        max_median = max_data * median_composit[column] # значение критерия 'mvax_median' = критерий max * медиану столбца
               
        if value < min_median  or value > max_median: # условие применения критериев
            df_zeros.at[i,column] = value  # запись 'подозрительного значения' в соотвествующую ячейку нолевой матрицы df_zeros
            

# Создадим строку медиан из hw_data_composite_analyzis и добавим ее в конце массива df_zeros!!!#new_row_median = hw_data_composite_analyzis.median(axis=0) # параметры новой строк в конце массива!!
#df_zeros.append(new_row_median, ignore_index = True) # создаем новую строку медиан

 
# pd.concat([df2, df])

In [ ]:
def analysis_dataset(df, min_data, max_data, median_composit):
# Функция analyzis_dataset создает массив 'подозрительных значений' в матрице заполненой NaN df_zeros
# Передем в функцию:
# df который будем исследовать
# min_data - минимальный коэффициент отсечки для выбросов значений "слева"
# max_data - максимальный коэффициент отсечки для выбросов значений "справа"
# значение критериев 'min_data'  - все значения меньше 10% от медианы столбца - 'подозрительное значение'
# значение критериев 'max_data'  - все значения больше медианы столбца в 10 раз  - 'подозрительное значение'
# принимаем min_data = 0.1
#           max_data = 10.0
# median_composit - список median массива df рассчитан ранее в программе
 
    list_df = list(df) # получаем список наименований столбцов из df
    
    # создаем df_zero заполненый 'NaN'  размерности  df (по умолчанию - с данными типа object)
    df_zeros = pd.DataFrame(columns=list_df, index = range(0, len(df)))
                      
    for column in df.columns: # иттерация по столбцам
        for i in range(len(df)): # иттерация по строкам
            
            value = df.at[i,column] # значение из df -  hw_data_composite_analyzis ВАЖНО писвоение .at[i,column]
        
            min_median = min_data * median_composit[column] # значение критерия 'min_median' = критерий min * медиану столбца
            max_median = max_data * median_composit[column] # значение критерия 'mvax_median' = критерий max * медиану столбца
               
            if value < min_median  or value > max_median: # условие применения критериев
                df_zeros.at[i,column] = value  # запись 'подозрительного значения' в соотвествующую ячейку нолевой матрицы df_zeros  
        
    # меняем тип данных  массива на float32                       
    df_zeros = df_zeros.astype(np.float32)
    
    return df_zeros

# Необходимо создать  отдельный модуль - функцию для проверки нашего DataSet 'hw_data_composite_analyzis' на 'подозрительные значения' и создание массива 'подозрительных значений'

In [ ]:
def analysis_dataset(df, min_data, max_data, median_composit):
# Функция analyzis_dataset создает массив 'подозрительных значений' в матрице заполненой нулями df_zeros
# Передем в функцию:
# df который будем исследовать
# min_data - минимальный коэффициент отсечки для выбросов значений "слева"
# max_data - максимальный коэффициент отсечки для выбросов значений "справа"
# значение критериев 'min_data'  - все значения меньше 10% от медианы- 'подозрительного значения'
# значение критериев 'max_data'  - все значения больше медианы в 10 раз  - 'подозрительного значения'
# принимаем min_data = 0.1
#           max_data = 10.0
# median_composit - список median массива df рассчитан ранее в программе

# создаем df_zero заполненый 'NaN'  размерности  df ( по умолчанию - с данными типа float64)
# list(df.columns) - это список наименований столбцов df (list)
# количество строк - len(df) 
    
    df_zeros = pd.DataFrame(columns = list(df.columns), index = range(0, len(df))) 

    for column in df.columns: # иттерация по столбцам
        for i in range(len(df)): # иттерация по строкам
            
            value = df.at[i,column] # значение из df -  hw_data_composite_analyzis ВАЖНО писвоение .at[i,column]
        
            min_median = min_data * median_composit[column] # значение критерия 'min_median' = критерий min * медиану столбца
            max_median = max_data * median_composit[column] # значение критерия 'mvax_median' = критерий max * медиану столбца
               
            if value < min_median  or value > max_median: # условие применения критериев
                df_zeros.at[i,column] = value  # запись 'подозрительного значения' в соотвествующую ячейку нолевой матрицы df_zeros  
                
    df_zeros = df_zeros.astype(np.float32)
    return df_zeros

# проверка функции

In [ ]:
df_zeros = analysis_dataset(hw_data_composite_analyzis, 0.1, 10, median_composit)  

In [ ]:
df_zeros.info()

In [ ]:
df_zeros.head(21).round(3)

как видим в строке с индексом 19 в первом и втором столбцах появились значения- маркеры- 0.01 - значит в этих ячейках были аномальные значения '0.0'. в других строках в других таблицах также стоят значения отличающиеся от ьедианы столбца. Например в строке 104 в столбце "модуль упругости, ГПа"

In [ ]:
df_zeros.iloc[100:110, 1 : 12].round(3) # !!! вывод необходимых строк

# у нас программа нашла, для примера 'подоззрительное значение' в строке 109 столбец 'модуль упругости, ГПа' = 9.986
и в строке 109 столбца 'Поверхностная плотность, г/м2' = 12.04

In [ ]:
df_zeros.nunique(axis= 0)

# Судя по данным .nunique(axis= 0) у нас всего 70 подозрительных значений в 4 колонках.

In [ ]:
# копируем df_zeros с ошибками в новый df и будем работать дальше с df_zeros_dropNaN
df_zeros_dropNaN = df_zeros.copy()
df_zeros_dropNaN

In [ ]:
# Удаляем все строки в которых содержаться исключительно значения NaN
# Мы получим df_analises в котором индексы укажут на строки в которых есть bugs!!!!!

df_analises = df_zeros_dropNaN.dropna(axis=0, how='all')

df_analises

In [ ]:
df_analises.info()

In [ ]:
# удаляем столбцы , где все значения NaN
df_analises.dropna(axis=1, how='all')

In [ ]:
# список индексов ,в которых есть  bugs. Таких строк  - 66!!!
list_id_bugs = list(df_analises.index) 
print(list_id_bugs)

# Рассчитаем количество строк в которых есть 'подозрительные значения' в массиве df_zeros.
Создадим массив строк df_analiz используя массив df_zeros, сохранив их оригинальные индексы в отдельном столбце и введя особый столбец с указнием количества 'подозрительных значений'
Если в какой-то строке будет больше 1 'подозрительноо значения', то  скорее всего мы будем удалять эту строку в нашем рабочем файле hw_data_composite_analyzis

Мы можем использовать метод .rename() для переименования определенных или всех столбцов с помощью словаря. Нам не нужны круглые скобки, поэтому давайте избавимся от них:

!!!Если вы будете работать с набором данных в течение некоторого времени, рекомендуется использовать нижний регистр, удалить специальные символы и заменить пробелы символами подчеркивания.